In [24]:
from langchain.chains.question_answering.map_reduce_prompt import messages
!pip install python-dotenv
!pip install -U openai langchain langchain-openai
!pip install langchain_community
!pip install pypdf
!pip install faiss-cpu

ModuleNotFoundError: No module named 'pydantic'

In [10]:
from dotenv import load_dotenv
import os
import openai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

In [11]:
def env_loader():
    load_dotenv()
    openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
def pdf_loader(file_path):
    loader = PyPDFLoader(file_path)
    return loader

In [13]:
def show_pdf(loader):
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    context = [doc.page_content for doc in docs]

    print(context)

In [14]:
def create_vectorstore(loader):
    embeddings = OpenAIEmbeddings()
    #embeddings = HuggingFaceEmbeddings()

    index = VectorstoreIndexCreator(
        vectorstore_cls=FAISS,
        embedding=embeddings,
        ).from_loaders([loader])

    # 파일로 저장
    index.vectorstore.save_local("vectorstore")

    return index

In [15]:
def set_model(model):

    chat = ChatOpenAI(
        model=model,
        temperature=0.3,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    return chat

In [19]:
def chat_setup():
    file_path = "C:/Users/user/Documents/GitLab/mammoth_Langchain/doc/타임소프트 회사소개서.pdf"

    env_loader()
    loader = pdf_loader(file_path)
    # show_pdf(loader)
    index = create_vectorstore(loader)
    chat = set_model(model='gpt-4o-mini')

    return chat, index

In [17]:
def chat_input(index, user_input, chat):
    return index.query( user_input, llm=chat, )

In [22]:
while True:
    chat, index = chat_setup()

    user_input = input("you: ")
    print(f"you: {user_input}")

    if user_input == "exit":
        break
    elif user_input.replace(" ", "") == "":
        print("chat: 질문이 없습니다. 정확한 질문을 해주시면 답변해 드리겠습니다.")
    else:
        chat_text = chat_input(
            chat=chat,
            index=index,
            user_input=user_input,
        )
        print(f"chat: {chat_text}")

you: exit
